# Processor

This python notebook consists of a set of functions that take a JSON input from the `scraper` tool, and get the data into a format that we can serve on the final website. 

## JSON to CSV

As our input data is currently formatted in JSON, we want to convert that to CSV so we can work with entire columns, and eventually push to a database like PostgreSQL.

To avoid corrupting the original data that was collected, place a copy of the data in the convenience folder `./_raw_json` in this directory, and only modify this copied data as you work with the modules in this notebook.

The following block of code will go through all the JSON files in the specified directory, flatten them, convert them into CSV's, and save them all to disk in the `./_raw_csv` directory.

**NOTE**: This is the only part of the process that's somewhat hardcoded. In order to combine the JSON into a single CSV, the column names have to be consistently named, and its easier to easier to acheive this by edit the JSON's before saving as CSV's.

In [1]:
%load_ext autoreload
%autoreload 2

import json
import os
import pandas as pd

from flatten_json import flatten
from tqdm import tqdm

JSON_DIR = './_raw_json'
CSV_DIR = './_raw_csv'
CSV_FINAL = 'all.csv'

#### Helper Functions

In [32]:
def delete_keys_from_dict(d, to_delete):
    if isinstance(to_delete, str):
        to_delete = [to_delete]
    if isinstance(d, dict):
        for single_to_delete in set(to_delete):
            if single_to_delete in d:
                del d[single_to_delete]
        for k, v in d.items():
            delete_keys_from_dict(v, to_delete)
    elif isinstance(d, list):
        for i in d:
            delete_keys_from_dict(i, to_delete)

#### Run

In [36]:
for filename in tqdm(os.listdir(JSON_DIR)):
    if ".json" in filename:
        with open(f'{JSON_DIR}/{filename}', 'r', encoding='utf-8') as f:
            data = json.load(f)
            
            # 1. First globally delete all the keys that we don't want.
            delete_keys_from_dict(data, ['__typename', 'best_datasheet', 'best_image', 'manufacturer_url'])
            
            for result in data['data']['search']['results']:
                # 2. Within the JSON, flatten the `specs` array from 
                #    'specs': [{'attribute': {'id': '548',
                #                             'name': 'Capacitance' 
                #                             'shortname': 'capacitance'
                #                             '__typename': 'Attribute'
                #                            },
                #               'display_value': '100 nF'
                #              },
                #              { ... },
                #              { ... },
                #              ...
                #             ]
                #    to
                #    'specs': {'capacitance': {'display_value': '100 nF', 'id': '548'},
                #              'case_package': {'display_value': 'Radial', 'id': '842'},
                #              'depth': {'display_value': '8 mm', 'id': '291'},
                #              ...
                #             }    
                #    and remove some fields that we don't want to include.
                spec_json = {}
                for spec in result['part']['specs']:
                    title = spec['attribute']['shortname']
                    spec['attribute']['display_value'] = spec['display_value']
                    spec = spec['attribute']
                    del spec['shortname']
                    del spec['name']
                    spec_json[title] = spec
                result['part']['specs'] = spec_json


                # 3. Remove specific parts of the JSON that we don't want (duplicate fields, etc).
                del result['part']['_cache_id']
                del result['part']['descriptions']
                del result['part']['counts']
            
            # 4. Run the `flatten` function on each of the parts, place it in a list, and convert 
            #    to a Pandas DF.
            flat = [flatten(d) for d in data['data']['search']['results']]
    
            df = pd.DataFrame(flat, dtype ='str')
            df.to_csv(f"{CSV_DIR}/{filename.split('.')[0]}.csv", index=False)


100%|███████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:12<00:00, 18.07s/it]


#### Combine

In [45]:
filenames = [f"{CSV_DIR}/{filename}" for filename in os.listdir(CSV_DIR) if "all" not in filename and ".csv" in filename]
df = pd.concat(map(pd.read_csv, filenames), ignore_index=True)
df = df.astype(str)
df.to_csv(f"{CSV_DIR}/{CSV_FINAL}", index=False)


['./_raw_csv/film.csv', './_raw_csv/mica.csv', './_raw_csv/ceramic.csv', './_raw_csv/aluminum_electrolytic.csv']


## CSV Postprocessing

In this step, we want to take the combined CSV that we generated in the previous step and format it into the final format that we will upload to the PostgreSQL database.

We will use a modular approach. For each step of updating the CSV, we will implement a function that takes in a pandas dataframe and outputs another pandas dataframe in the desired format. 

In [5]:
df = pd.read_csv(f'{CSV_DIR}/all.csv', index_col=False)

/var/folders/21/pp5p0yss3xq_1z5gvfm0shgh0000gn/T/ipykernel_67893/3566771438.py:1: DtypeWarning: Columns (14,16,18,24,26,28,30,36,38,40,44,46,48,50,54,58,60,62,64,66,70,72,74,78,80,82,84,86,88,90,92,94,96,99,101,103,107,109,115,117,119,121,123,125,127,129,131,133,135,141,143,145,147,149,153,155,157,159,161,163,165,167,169,171,173,175,177,179,181,183,185,187,189,191,193,195,197,199,201,203,205,207,209,213,217,219,221,223,225,227,229,231,235,237,239,243,245,247,249,251,253,255,257,259,261,263,265,267,269,271,273,275,277,279,281,283,285,289,291,293,295,297,299,301,303,305,309,311,313,315,317) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{CSV_DIR}/all.csv', index_col=False)


In [6]:
string_to_float_cols = [
    'part_specs_tolerance_display_value', 
    'part_specs_temperaturecoefficient_display_value', 
    'part_specs_maxjunctiontemperature_display_value', 
    'part_specs_maxoperatingtemperature_display_value', 
    'part_specs_minoperatingtemperature_display_value', 
    'part_specs_dissipationfactor_display_value', 
    'part_specs_failurerate_display_value', 
    'part_specs_frequencytolerance_display_value', 
    'part_specs_qfactor_display_value', 
    'part_specs_frequencystability_display_value', 
    'part_specs_viewingangle_display_value', 
    'part_specs_accuracy_display_value', 
    'part_specs_speedgrade_display_value', 
    'part_specs_inductancetolerance_display_value', 
    'part_specs_ambienttemperaturerangehigh_display_value'
]

string_to_int_cols = [
    'part_specs_numberofpins_display_value', 
    'part_specs_life_hours__display_value', 
    'part_specs_numberofelements_display_value', 
    'part_specs_numberofterminals_display_value', 
    'part_specs_numberofchannels_display_value', 
    'part_specs_life_cycles__display_value', 
    'part_specs_commonmoderejectionratio_display_value', 
    'part_specs_gainbandwidthproduct_display_value', 
    'part_specs_numberofcapacitors_display_value', 
    'part_specs_numberofcircuits_display_value', 
    'part_specs_numberofi_os_display_value', 
    'part_specs_numberofleds_display_value', 
    'part_specs_numberofpositions_display_value'
]

string_to_base_float_cols = [
    'part_specs_capacitance_display_value', 
    'part_specs_depth_display_value', 
    'part_specs_height_display_value', 
    'part_specs_height_seated_max__display_value', 
    'part_specs_leaddiameter_display_value', 
    'part_specs_leadpitch_display_value', 
    'part_specs_length_display_value', 
    'part_specs_voltage_display_value', 
    'part_specs_voltagerating_display_value', 
    'part_specs_voltagerating_ac__display_value', 
    'part_specs_voltagerating_dc__display_value', 
    'part_specs_width_display_value', 
    'part_specs_weight_display_value', 
    'part_specs_leadlength_display_value', 
    'part_specs_insulationresistance_display_value', 
    'part_specs_diameter_display_value', 
    'part_specs_thickness_display_value', 
    'part_specs_esr_equivalentseriesresistance__display_value', 
    'part_specs_pitch_display_value', 
    'part_specs_resistance_display_value', 
    'part_specs_dcresistance_dcr__display_value', 
    'part_specs_inductance_display_value', 
    'part_specs_maxdccurrent_display_value', 
    'part_specs_powerrating_display_value', 
    'part_specs_seriesresistance_display_value', 
    'part_specs_terminalpitch_display_value', 
    'part_specs_currentrating_display_value', 
    'part_specs_characterheight_display_value', 
    'part_specs_wire_cablediameter_display_value', 
    'part_specs_ripplecurrent_display_value',
    'part_specs_cablelength_display_value', 
    'part_specs_maxsupplyvoltage_display_value', 
    'part_specs_minsupplyvoltage_display_value', 
    'part_specs_voltagegain_display_value', 
    'part_specs_loadcurrent_display_value', 
    'part_specs_outputcurrent_display_value', 
    'part_specs_maxlength_display_value', 
    'part_specs_maxthickness_display_value', 
    'part_specs_maxwidth_display_value', 
    'part_specs_minlength_display_value', 
    'part_specs_minthickness_display_value', 
    'part_specs_minwidth_display_value', 
    'part_specs_insidediameter_display_value', 
    'part_specs_responsetime_display_value', 
    'part_specs_nominalsupplycurrent_display_value', 
    'part_specs_selfresonantfrequency_display_value', 
    'part_specs_memorysize_display_value', 
    'part_specs_current_display_value', 
    'part_specs_maxcurrentrating_display_value', 
    'part_specs_maxvoltagerating_dc__display_value', 
    'part_specs_maxoutputcurrent_display_value', 
    'part_specs_mininputvoltage_display_value', 
    'part_specs_outputvoltage_display_value', 
    'part_specs_bandwidth_display_value', 
    'part_specs_nominalsupplyvoltage_dc__display_value', 
    'part_specs_dropoutvoltage_display_value', 
    'part_specs_terminalwidth_display_value', 
    'part_specs_forwardcurrent_display_value', 
    'part_specs_forwardvoltage_display_value', 
    'part_specs_maxrepetitivereversevoltage_vrrm__display_value', 
    'part_specs_peaknon_repetitivesurgecurrent_display_value', 
    'part_specs_peakreversecurrent_display_value', 
    'part_specs_maxfrequency_display_value', 
    'part_specs_ramsize_display_value', 
    'part_specs_leakagecurrent_display_value', 
    'part_specs_testfrequency_display_value', 
    'part_specs_ripplecurrent_ac__display_value', 
    'part_specs_impedance_display_value', 
    'part_specs_holediameter_display_value', 
    'part_specs_outsidediameter_display_value', 
    'part_specs_switchingvoltage_display_value',
    'part_specs_workingvoltage_display_value', 
    'part_specs_loadcapacitance_display_value', 
    'part_specs_operatingsupplyvoltage_display_value', 
    'part_specs_frequency_display_value', 
    'part_specs_databuswidth_display_value'
    'part_specs_luminousintensity_display_value', 
]

In [7]:
%aimport compute
df = compute.spec_string_to_float(df, cols=string_to_float_cols)
df = compute.spec_string_to_int(df, cols=string_to_int_cols)
df = compute.spec_string_to_base_float(df, cols=string_to_base_float_cols)

"""
The total list of dielectric types from our dataset is:
{'Z5F', 'Z5U', 'X8L', 'P2H', 'K2000', 'X7T', 'X0U', 'Y5F', 'X7S', 'X5F', 'Z5T', 'X5R', 'Y5R', 'C0J', 'Y5P', 'X5S', 'S2H', 'Z5P', 'K4000', 'Z7S', 'X6S', 'X8R', 'R3L', 'PP', 'C0H', 'Y5T', 'X5P', 'Y5U', 'R2H', 'Y5V', 'X8G', 'PPS', 'C0G', 'U2J', 'M3K', 'Z5V', 'T3M', 'PET', 'X7R', 'S3N', 'X7U', 'Y5E', 'P3K', 'C0K', 'T2H', 'Mica', 'Y5S', 'X6T', 'X5U', 'NP0'}
"""
df = compute.classify_ceramic(df)

In [8]:
df.to_csv("test.csv", index=False)

In [14]:
df2 = compute.compute_volume(df)
df2.to_csv("test_vol.csv", index=False)

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
359969   NaN
359970   NaN
359971   NaN
359972   NaN
359973   NaN
Name: volume, Length: 359974, dtype: float64
